In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')
import time
import random
from tqdm import tqdm_notebook

# Fastai
from fastai import *
from fastai.tabular import *

plt.style.use('seaborn')
sns.set(font_scale=1)

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split

In [2]:
random_state = 3710
np.random.seed(random_state)
tr = pd.read_csv('/storage/santander_comp/train_freq.csv')
te = pd.read_csv('/storage/santander_comp/test_freq.csv')

In [ ]:
tr_small = tr.sample(frac=0.15)

In [ ]:
print("% of target in small train == 1: {:01.2f}%".format(len(tr_small.loc[tr_small['target'] == 1]) / len(tr_small) * 100))
print("% of target in train == 1: {:01.2f}%".format(len(tr.loc[tr['target'] == 1]) / len(tr) * 100))

In [ ]:
features = [feature for feature in tr.columns if feature not in ['ID_code', 'target']]


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(tr[features],
                                                  tr['target'],
                                                  test_size = 0.20,
                                                  random_state = random_state)

In [ ]:
# param_grid = {
# #     "bagging_fraction" : [0.3, 0.5],
# #     "bagging_freq": [3, 5],
# #     "feature_fraction" : [0.03, 0.04],
#       "learning_rate" : [0.0005, 0.003, 0.007, 0.008, 0.01, 0.02],
# #     "min_data_in_leaf": [5, 25, 60, 80, 100, 150],
# #     "num_leaves" : [2, 20, 100, 1000],
# }

In [ ]:
# mdl = lgb.LGBMClassifier(boosting_type= 'gbdt',
#           objective = 'binary',
#           metric = 'auc',
#           num_leaves = params['num_leaves'],
#           num_threads = params['num_threads'],
#           learning_rate = params['learning_rate'],               
#           bagging_freq = params['bagging_freq'],
#           bagging_fraction = params['bagging_fraction'],
#           feature_fraction = params['feature_fraction'],
#           min_data_in_leaf = params['min_data_in_leaf'],
#           min_sum_hessian_in_leaf = params['min_sum_hessian_in_leaf'],
#           tree_learner = params['tree_learner'],
#           boost_from_average = params['boost_from_average'],
# #           bagging_seed = params['bagging_seed'],
#           verbosity = params['verbosity'],
#           seed = params['seed'])


In [ ]:
trn_idx = X_train.index
val_idx = X_val.index

In [ ]:
params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting_type": 'gbdt',
    "num_leaves" : 13,
    "learning_rate" : 0.0085,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.04,
    "min_data_in_leaf": 80,
    "min_sum_hessian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    "verbosity" : -1,
    "seed": random_state
}

In [ ]:
oof = tr[['ID_code', 'target']]
oof['predict'] = 0
predictions = tr[['ID_code']]
val_aucs = []
feature_importance_df = pd.DataFrame()
p_val, yp = 0, 0
trn_data = lgb.Dataset(X_train, label = y_train)
val_data = lgb.Dataset(X_val, label = y_val)
evals_result = {}
lgb_clf = lgb.train(params,
                        trn_data,
                        100000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=2000,
                        verbose_eval = 1000,
                        evals_result=evals_result
                       )

p_val += lgb_clf.predict(X_val)
feature_importance_df["feature"] = features
feature_importance_df["importance"] = lgb_clf.feature_importance()
oof['predict'][val_idx] = p_val
val_score = roc_auc_score(y_val, p_val)
val_aucs.append(val_score)
predictions = yp

In [6]:
features = [feature for feature in tr.columns if feature not in ['ID_code', 'target']]


In [7]:
X_train, X_val, y_train, y_val = train_test_split(tr[features],
                                                  tr['target'],
                                                  test_size = 0.20,
                                                  random_state = 42)

In [8]:
params = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting_type": 'gbdt',
    "learning_rate": 0.01,
    "num_leaves" : 16,
    "bagging_freq": 5,
    "bagging_fraction" : 0.333,
    "feature_fraction" : 0.041,
    "min_data_in_leaf": 140,
    "min_sum_hessian_in_leaf": 10,
    "tree_learner": "serial",
    "boost_from_average": "false",
    "verbosity" : -1,
    "seed": random_state
}

In [9]:
feature_importance_df = pd.DataFrame()

In [10]:
trn_data = lgb.Dataset(X_train, label = y_train)
val_data = lgb.Dataset(X_val, label = y_val)
evals_result = {}
lgb_clf = lgb.train(params,
                        trn_data,
                        100000,
                        valid_sets = [trn_data, val_data],
                        early_stopping_rounds=2000,
                        verbose_eval = 1000,
#                         learning_rates=lambda iter: 0.006 * (0.97 ** iter),
                        evals_result=evals_result
                       )


feature_importance_df["feature"] = features
feature_importance_df["importance"] = lgb_clf.feature_importance()


Training until validation scores don't improve for 2000 rounds.
[1000]	training's auc: 0.908953	valid_1's auc: 0.885829
[2000]	training's auc: 0.920456	valid_1's auc: 0.893866
[3000]	training's auc: 0.928328	valid_1's auc: 0.897185
[4000]	training's auc: 0.934585	valid_1's auc: 0.899009
[5000]	training's auc: 0.940149	valid_1's auc: 0.900086
[6000]	training's auc: 0.945277	valid_1's auc: 0.900635
[7000]	training's auc: 0.950015	valid_1's auc: 0.900774
[8000]	training's auc: 0.954475	valid_1's auc: 0.900795
[9000]	training's auc: 0.958661	valid_1's auc: 0.900737
[10000]	training's auc: 0.962585	valid_1's auc: 0.900796
Early stopping, best iteration is:
[8398]	training's auc: 0.956163	valid_1's auc: 0.900892
